In [1]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

#Use conda_aamzonei_mxnet_p36 Kernel - Otherwise we should pip install this

  Using cached https://files.pythonhosted.org/packages/fa/7d/5042b668a8ed41d2a80b8c172f5efcd572e3c046c75ae029407e19b7fc68/opencv_python-3.4.2.16-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: opencv-python 3.4.2.17
    Uninstalling opencv-python-3.4.2.17:
      Successfully uninstalled opencv-python-3.4.2.17
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/08/f1/66330f4042c4fb3b2d77a159db8e8916d9cdecc29bc8c1f56bc7f8a9bec9/opencv_contrib_python-3.4.2.16-cp36-cp36m-manylinux1_x86_64.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import s3fs
from PIL import Image
import cv2

from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np
import os

In [3]:
# Create SIFT Object
sift = cv2.xfeatures2d.SIFT_create()

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)  

flann = cv2.FlannBasedMatcher(index_params,search_params)

In [4]:
fs = s3fs.S3FileSystem()
processed_combo=[]

bucket_name='sagemaker-aiml-zsl/'
folder_name='DeDuplication-Images/'

for i in fs.ls('s3://'+bucket_name+folder_name):
  #Set key name
  i1_name=i
  #Read from S3
  f1=fs.open(f's3://'+i)
  img1=Image.open(f1)
  image1_data = np.asarray(img1)  
    
  kp1_temp, des1_temp = sift.detectAndCompute(image1_data,None)

  for j in fs.ls('s3://sagemaker-aiml-zsl/DeDuplication-Images/'):
     #Set key name
     i2_name=j
    
     #Checks whether the combo is already processed, if yes it wont allow inside   
     if (i1_name,i2_name) not in processed_combo:
        #Read from S3
        f2=fs.open(f's3://'+j)
        img2=Image.open(f2)
        image2_data = np.asarray(img2)

        print(i1_name,i2_name)

        if i != j:
          kp2_temp, des2_temp = sift.detectAndCompute(image2_data,None)
          #K nearest matches
          matches_temp = flann.knnMatch(des1_temp,des2_temp,k=2)

          good_temp = []
          number_keypoints_temp = 0

          # ratio test
          for match1,match2 in matches_temp:
            if match1.distance < 0.7*match2.distance:
              good_temp.append([match1])

          #Check features count - To delete based on that
          if len(kp1_temp) >= len(kp2_temp):
            number_keypoints_temp = len(kp1_temp)
            del_img= i2_name
          else:
            number_keypoints_temp = len(kp2_temp)
            del_img= i1_name
          print(number_keypoints_temp,len(good_temp))

          #Find sililarity percentage
          percentage_similarity = len(good_temp) / number_keypoints_temp * 100
          print("Similarity: " + str(int(percentage_similarity)) )

          #Deletion criteria
          if int(percentage_similarity) > 3:
            print('Images ' +i1_name+ ' and ' +i2_name+ ' are duplicates and we can delete '+del_img + "\n" )
          else:
            print('Images ' +i1_name+ ' and ' +i2_name+ ' are not duplicates' + "\n")

          #Save the 2 images combo as tuple and add this to list
          tup1=(i1_name,i2_name)
          tup2=(i2_name,i1_name)
          processed_combo.append(tup1)
          processed_combo.append(tup2)
      
      

sagemaker-aiml-zsl/DeDuplication-Images/IMG-3254.jpg sagemaker-aiml-zsl/DeDuplication-Images/IMG-3254.jpg
sagemaker-aiml-zsl/DeDuplication-Images/IMG-3254.jpg sagemaker-aiml-zsl/DeDuplication-Images/IMG-3255.jpg
105133 18903
Similarity: 17
Images sagemaker-aiml-zsl/DeDuplication-Images/IMG-3254.jpg and sagemaker-aiml-zsl/DeDuplication-Images/IMG-3255.jpg are duplicates and we can delete sagemaker-aiml-zsl/DeDuplication-Images/IMG-3255.jpg

sagemaker-aiml-zsl/DeDuplication-Images/IMG-3254.jpg sagemaker-aiml-zsl/DeDuplication-Images/IMG-3530.jpg
105133 62
Similarity: 0
Images sagemaker-aiml-zsl/DeDuplication-Images/IMG-3254.jpg and sagemaker-aiml-zsl/DeDuplication-Images/IMG-3530.jpg are not duplicates

sagemaker-aiml-zsl/DeDuplication-Images/IMG-3255.jpg sagemaker-aiml-zsl/DeDuplication-Images/IMG-3255.jpg
sagemaker-aiml-zsl/DeDuplication-Images/IMG-3255.jpg sagemaker-aiml-zsl/DeDuplication-Images/IMG-3530.jpg
91897 54
Similarity: 0
Images sagemaker-aiml-zsl/DeDuplication-Images/IMG-325